In [1]:
using Revise

using RandomPositions
using Distributions
using Rotations
using CSV
using DataFrames
using Plots

┌ Info: Precompiling RandomPositions [3f373569-fa07-4ec2-97f5-a52c11e59be5]
└ @ Base loading.jl:1317


In [21]:
using Statistics: mean

In [3]:
k_b = 1.3806e-23
m_Rb = 1.6605e-27 * 87
T = 60e-6

w_x = 198. * sqrt(2)
w_yz = 110.
n_gs = round(100)

params = RydbergParams(
    distribution = MvNormal([10., 10., 10.]), #MvNormal([w_x; w_yz; w_yz]),
    n_gs = n_gs,
    rot = RotZ(2π/360 * -45.0),
    t_exc = 10.0,
    Ω_c_0 = 1.6/(2π),
    σ_c = 60.0,
    pos_c = [0.0; 0.0],
    Ω_p_0 = 8.83/(2π),
    σ_p = 1500.0,
    pos_p = [0.0; 0.0],
    Γ_p = 6.0,
    Δ_1γ = 97.0,
    Δ_2γ = 0.0,
    γ = 0.0,
    C6 = 60000.0,
    pulse_shape = 100.,  # 'Shape of the excitation pulse': objects=['rect', 'gauss', 'sech2', 'lorentz']
    laser_width = 0.0
)

vdw_params = PowerLaw(6, 2π*267000)

PowerLaw
  α: Float64 6.0
  C: Float64 1.6776104770169496e6


In [53]:
t_exc = 1.0
Ω_c_0 = 13.0
σ_c = 70.0
pos_c = [0.0; 0.0]
Ω_p_0 = 1.3
σ_p = 100.0
pos_p = [0.0; 0.0]
Γ_p = 6.0
Δ_1γ = 97.0
Δ_2γ = 0.0
γ = 0.0
C6 = 9300.0
pulse_shape = :rect  # 'Shape of the excitation pulse': objects=['rect', 'gauss', 'sech2', 'lorentz']
laser_width = 0.14

gs_pos = rand(UniformBallSampler(3, 50.), 10000)

ge_pos = mask_center(gs_pos, σ_p, pos_c=pos_p, dims=[1, 2])
ge_pos = mask_center(gs_pos, σ_c, pos_c=pos_c, dims=[2, 3])

Ω_p = get_Ω(ge_pos, Ω_p_0, σ_p, pos_c=pos_p, dims=[1, 2])
Ω_c = get_Ω(ge_pos, Ω_c_0, σ_c, pos_c=pos_c, dims=[2, 3])
Ω_2γ =  Ω_c .* Ω_p ./ (2 * Δ_1γ)
σ_fourier = 1.
Rc = get_Rc(ge_pos, Ω_2γ, σ_fourier, laser_width, C6)
println("minimum Rc: $(minimum(Rc)), maximum Rc: $(maximum(Rc))")
Nb = get_Nb_from_Rc(ge_pos, Rc)
Rc .= 10.
println("minimum Nb: $(minimum(Nb)), maximum Nb: $(maximum(Nb))")

Ω_eff = sqrt.(Nb) .* Ω_2γ
ρ = get_ρ.(Ω_eff, Δ_2γ, t_exc);
println("mean ρ: $(mean(ρ))")

minimum Rc: 4.575548800229092, maximum Rc: 4.57703668647595
minimum Nb: 1, maximum Nb: 24
mean ρ: 0.39160460418601334


In [54]:
# pos = pick_hard_spheres(ge_pos, ρ, Rc)
pos = pick_hard_spheres(ge_pos, NonuniformPicking(ρ), FixedNStop(1000), NonuniformBlockade(Rc))

┌ Warning: Already fully blockaded. Desired number of rydbergs was not reached
└ @ RandomPositions d:\Dropbox\Rydberg\Software\Julia\RandomPositions\src\hard_sphere.jl:69


3×298 Matrix{Float64}:
 -18.6932   -4.13875  -18.0657  -33.0637   …  -25.1893  -15.605   -38.8327
  18.0621   -8.76104   22.76     -7.04686     -16.9017  -15.971    -0.158631
 -27.032   -19.2114   -11.8016   -4.37743      32.0986   16.8947   29.5359

In [55]:
limits = (-50, 50)
scatter(
    pos[1, :], pos[2, :], pos[3, :],
    xlims=limits, ylims=limits,zlims=limits,
    markersize=1
)

<!DOCTYPE html>
 
 
 Plots.jl